# Neo4J Friends Graph


Dependecies:

`pip install neo4j`

`pip install neo4j-driver`

`pip install neobolt`

`pip install neo4jupyter` 

`pip install py2neo` 

References: https://neo4j.com/developer/

In [2]:
!pip install neo4j

!pip install neo4j-driver

!pip install neobolt

!pip install neo4jupyter 

!pip install py2neo

     |████████████████████████████████| 174 kB 3.2 MB/s eta 0:00:01
  Created wheel for neo4j: filename=neo4j-5.2.1-py3-none-any.whl size=249276 sha256=f34ef1555b3a70f61c16b9122f78c39697c50585f61c1df8a5a087ca42b07de5
  Stored in directory: /Users/lmschwenke/Library/Caches/pip/wheels/10/2e/22/b2aa68e1d39d749de7c3d8592ae34561f16b851499ad1b5256
Successfully built neo4j
     |████████████████████████████████| 174 kB 4.8 MB/s eta 0:00:01
  Created wheel for neo4j-driver: filename=neo4j_driver-5.2.1-py3-none-any.whl size=249371 sha256=4a2dbd7d66f17f00f36aafe0d8c771f44bd1eb48aab0cb7fcdc798de8fc1134c
  Stored in directory: /Users/lmschwenke/Library/Caches/pip/wheels/4b/d4/67/5db735369dd2a803d34bbb4ffa7528d184bfb42ebe470db6c9
Successfully built neo4j-driver
     |████████████████████████████████| 183 kB 4.7 MB/s eta 0:00:01
  Created wheel for neobolt: filename=neobolt-1.7.17-py3-none-any.whl size=37180 sha256=9bd3ca9cf05babed5fa14f8953f9f3f60b443b44431810ba10a06f7f76622c66
  Stored in director

  Created wheel for neo4jrestclient: filename=neo4jrestclient-2.1.1-py2.py3-none-any.whl size=78563 sha256=8f8bde358bcad9cdd8c53be497306ff8a4ba80e984fd3b452b8f99074d288046
  Stored in directory: /Users/lmschwenke/Library/Caches/pip/wheels/e7/0c/52/6b804635a6a6a03b3d63810077e454af8ba96604b0d5a83441
Successfully built neo4jupyter ipython-cypher neo4jrestclient


### Create Neo4J Connection

In [3]:
from neo4j import GraphDatabase, basic_auth

In [9]:
url = "bolt://localhost:7687"
user = "neo4j"
password = "neo4j!"

driver = GraphDatabase.driver(url, auth=basic_auth(user, password))
session = driver.session()
session

### Create Friends Graph

In [5]:
# Insert data
# UNWIND expands a list into a sequence of rows. 

insert_query = '''
UNWIND $pairs as pair
MERGE (p1:Person {name:pair[0]})
MERGE (p2:Person {name:pair[1]})
MERGE (p1)-[:KNOWS]-(p2);
'''

In [10]:
data = [["Jim","Mike"],["Jim","Billy"],["Anna","Jim"],
          ["Anna","Mike"],["Sally","Anna"],["Joe","Sally"],
          ["Joe","Bob"],["Bob","Sally"]]

session.run(insert_query, parameters={"pairs": data})

ServiceUnavailable: Couldn't connect to localhost:7687 (resolved to ()):
Failed to establish connection to ResolvedIPv6Address(('::1', 7687, 0, 0)) (reason [Errno 61] Connection refused)
Failed to establish connection to ResolvedIPv4Address(('127.0.0.1', 7687)) (reason [Errno 61] Connection refused)

### Visualize Graph

In [5]:
import neo4jupyter
neo4jupyter.init_notebook_mode()

from py2neo import Graph
g = Graph(url,auth=(user, password))
   
neo4jupyter.draw(g, options = {"Person": "name"} )

<IPython.core.display.Javascript object>

### Queries

In [6]:
# Friends of a friend

foaf_query = '''
MATCH (person:Person)-[:KNOWS]-(friend)-[:KNOWS]-(foaf) 
WHERE person.name = $name
  AND NOT (person)-[:KNOWS]-(foaf)
RETURN foaf.name AS name
'''

results = session.run(foaf_query, parameters={"name": "Joe"})
for record in results:
    print(record["name"])

Anna


In [7]:
# Common friends

common_friends_query = """
MATCH (user:Person)-[:KNOWS]-(friend)-[:KNOWS]-(foaf:Person)
WHERE user.name = $user AND foaf.name = $foaf
RETURN friend.name AS friend
"""

results = session.run(common_friends_query, parameters={"user": "Jim", "foaf": "Mike"})
for record in results:
    print(record["friend"])


Anna


In [8]:
# Connecting paths
connecting_paths_query = """
MATCH path = shortestPath((p1:Person)-[:KNOWS*..6]-(p2:Person))
WHERE p1.name = $name1 AND p2.name = $name2
RETURN path
"""

results = session.run(connecting_paths_query, parameters={"name1": "Joe", "name2": "Billy"})
for record in results:
    print (record["path"])

session.close()

<Path start=<Node id=29 labels=frozenset({'Person'}) properties={'name': 'Joe'}> end=<Node id=32 labels=frozenset({'Person'}) properties={'name': 'Billy'}> size=4>
